# Heading level 1
#Heading level 2
**This is a highlight**
*This is italic*
~strick~
***Bold italalic***


<a href="https://colab.research.google.com/github/gisalgs/notebooks/blob/main/POI-case-study-part1-v2-colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Case Study: Accessibility - Part 1

The City of Columbus has put together a collection of points of interest for different functional categories. This data can be viewed at their interactive map https://opendata.columbus.gov/datasets/columbus::points-of-interest/explore. The description of the data can be found at https://maps2.columbus.gov/arcgis/rest/services/Schemas/PointsOfInterest/MapServer/10. The raw geojson file has been posted at https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/columbus_points_of_interest.geojson, which can also be downloaded from their interactive map page. We will use this geojson file in this tutorial.

The goal of this tutorial is to demonstrate the use of Python for spatial data analysis. More specifically, we will examine how residents in Franklin county can access some of the services represented in this data set. Of course the data is probably incomplete because it may not fully cover areas outside the city of Columbus and some points of interest may not be included in the data either. The main purpose is to demonstrate the use of data and Python coding to analyze accessibility.

It is best to treat this tutorial as a working notebook (it is indeed a Jupyter Notebook). Only the code for the beginning is included. Students will need to type the code by following the video or in-class instruction. Markdown cells should also be used to explain the code.

After finishing this tutorial, students should answer the following questions (with code and results from the code):

1. How many types of points of interest are represented in the data and how many instances are there in each type?
2. How the POIs are distributed on a map?

## Environment and Data

Since we are going to run every piece of code on a Jupyter Notebook or Jupyter Lab server, we get direct access to the cloud. Luckily everything we use, the data and library will all be available in the cloud.

First, let's make sure we have the geometry module used so far in this class cloned in our current working environment. After the following two lines of code, a colder called geom will be created and every file in the geom repo will be cloned to that folder. This allows us to directly use the geom modules without any additional process.

In [1]:
!rm -rf geom
!git clone https://github.com/gisalgs/geom.git

Cloning into 'geom'...
remote: Enumerating objects: 375, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 375 (delta 27), reused 6 (delta 2), pack-reused 314 (from 1)
Receiving objects: 100% (375/375), 98.96 KiB | 4.71 MiB/s, done.
Resolving deltas: 100% (204/204), done.


In [10]:
from geom.point import *

The following two libraries are essential to access online data files and handle JSON and GeoJSON files. Here we will use a more popular (and more stable) module called `requests` (in 2025) to access online files.

In [12]:
# import urllib.request as request
import requests
import json

### Population data

In addition to the POIs, we will also need to have the population data at the census block group level for Franklin county.

In [14]:

import requests
import json
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/blockgrps_pop_franklin_2.geojson'

blkgrps = json.loads(requests.get(url).text)

len(blkgrps['features'])

887

We can also try the `urllib` module as follows, but it may run into some issues on macOS.

```python
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/blockgrps_pop_franklin_2.geojson'
with request.urlopen(url) as response:
    blkgrps = json.loads(response.read())

len(blkgrps['features'])
```

In [15]:
blkgrps['features'][0]['properties'].keys()

dict_keys(['GEO_ID', 'STATE', 'COUNTY', 'TRACT', 'BLKGRP', 'NAME', 'Franklin_2', 'HispPct', 'Total', 'White', 'WhitePct', 'Black', 'BlkPct', 'Asian', 'AsianPct'])

We are probably very eager to check how many of these census block group polygons are multiplygons. Actually all of them are!

In [16]:

sum([f['geometry']['type'] == 'MultiPolygon' for f in blkgrps['features']])

887

So how many are actually having multiple parts? There are only two.

In [17]:
sum([len(f['geometry']['coordinates'])>1 for f in blkgrps['features']])

2

Finally, are there any polygons (or multipolygons) with holes? There is actually one. So things will get complicated when we do drawing, etc.

In [18]:
sum([len(f['geometry']['coordinates'][0])>1 for f in blkgrps['features']])

1

Let's print out some info about this one.

In [19]:
for f in blkgrps['features']:
    if len(f['geometry']['coordinates'][0])>1:
        print(f['properties']['TRACT'], f['properties']['BLKGRP'])

008230 1


We will use the population data next time.

### POI data

Now we get the POI data, which can be either from the github data repo as below. Or we can download the data from the city's link and save it to an online location that is publicly accessible and use the link from there.

In [20]:
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/columbus_points_of_interest.geojson'

poi = json.loads(requests.get(url).text)

len(poi['features'])

15940

Explore the types of geometry in the POI data.

In [21]:
geom_types = []
for p in poi['features']:
    t = p['geometry']['type']
    if t not in geom_types:
        geom_types.append(t)

geom_types

['Point']

Now we know all the features are of the same type: Point. Let's know explore the attributes a little bit before we draw the map.

In [22]:
p['properties']

{'OBJECTID': 18901355,
 'COLS_KEY': 644807,
 'LSN': '2608 CLEVELAND AVE',
 'POI_NAME': 'Madina Auto Sales',
 'POI_TYPE': 'Retail - Commercial/Retail',
 'POI_SOURCE': None,
 'PRIMARY_USE': 'Y',
 'ALT_GEOM': 'N',
 'PHONE_NUM': None,
 'GLOBALID': '{F8459369-0754-4A0E-BFA0-191384166F76}',
 'WEBSITE': None,
 'OB_GYN': None,
 'PEDIATRICS': None,
 'PRIMARY_CARE': None,
 'SUTQ_RATING': None,
 'SNAP': None,
 'WIC': None}

#### Getting the types



In [24]:
poi_types = {}
for p in poi['features']:
    t = p['properties']['POI_TYPE']
    if t not in poi_types:
        poi_types[t] = 1
    else:
        poi_types[t] += 1

print('Total number of types:', len(poi_types))
for t in poi_types:
    print(f'{t:>40}: {poi_types[t]}')

Total number of types: 87
                  Industrial - Warehouse: 120
              Retail - Commercial/Retail: 2773
                         Medical - Other: 594
                        Medical - Dental: 154
                       Retail - Services: 2252
                     Medical - Physician: 297
                Medical - Women's Health: 47
                    Medical - Veterinary: 52
                   Public Places - Other: 74
                     Office - Non-Profit: 207
                      Office - Corporate: 891
        Public Places - House of Worship: 693
           Medical - Research/Laboratory: 32
                           Office - Bank: 172
                  Transportation - Other: 82
    Public Places - Theater/Concert Hall: 44
              Industrial - Manufacturing: 188
                      Retail - Mixed Use: 27
Public Places - Community/Recreation Center: 63
   Retail - Shopping Center-Neighborhood: 148
                       Education - Other: 96
            

Make sure to follow the instructions to complete the remaining of this notebook.

In [28]:
poi_types_main = {}
for t in poi_types:
    tt=t.split(' - ')[0]
    if tt not in poi_types_main:
        poi_types_main[tt] = 1
    else:
        poi_types_main[tt] += 1



In [29]:
print(len(poi_types_main))

17


In [30]:
poi_types_main

{'Industrial': 10,
 'Retail': 18,
 'Medical': 8,
 'Public Places': 13,
 'Office': 4,
 'Transportation': 5,
 'Education': 8,
 'Emergency Response': 3,
 'Government': 7,
 'Group Quarters': 4,
 'Retail – Specialty Food': 1,
 'Retail – Commercial/Retail': 1,
 'Retail – Grocery': 1,
 'Retail – Gas Station': 1,
 'Retail – Restaurant': 1,
 'Retail – Convenience Stores': 1,
 'Emergency  Response': 1}

In [32]:
for c in 'Retail – Specialty Food':
    print(ord(c), end=' ' if ord(c)<128 else f'({c}) ')

82 101 116 97 105 108 32 8211(–) 32 83 112 101 99 105 97 108 116 121 32 70 111 111 100 

In [34]:
chr(8211)

'–'

In [ ]:
import re

In [37]:
import re
poi_types_main = {}
for t in poi_types:
  tt = re.split(f' - | {chr(8211)} ', t)[0]
  if tt not in poi_types_main:
    poi_types_main[tt] = 1
  else:
    poi_types_main[tt] += 1

In [38]:
len(poi_types_main)

11

In [39]:
poi_types_main

{'Industrial': 10,
 'Retail': 24,
 'Medical': 8,
 'Public Places': 13,
 'Office': 4,
 'Transportation': 5,
 'Education': 8,
 'Emergency Response': 3,
 'Government': 7,
 'Group Quarters': 4,
 'Emergency  Response': 1}